# Data Science 101 - Part 2 of 2
## [Classifier Models](https://scikit-learn.org/stable/modules/tree.html#classification)

We'll use [scikit-learn](https://scikit-learn.org/stable/) to create a model that classifies some data (just created randomly) into 3 classes. This will be a good mockup of our cGMP datasets for the biosciences.

A classifier model separates 2 or more classes. For example, the blue and red lines below separate the filled circles from the empty circles. Each line would be a unique classifier (some are better than others as you can see).

![classifier image](https://upload.wikimedia.org/wikipedia/commons/thumb/2/20/Svm_separating_hyperplanes.png/1024px-Svm_separating_hyperplanes.png)

## Training a machine learning model to predict the class of a sample

We'll use [scikit-learn](https://scikit-learn.org) (aka sklearn) and [Dask](https://www.dask.org/) to build and train our classifier model.

### Scikit-learn - The Data Scientist's pocket knife

There are essentially 4 steps to building a model with scikit-learn:
1. Build the dataloader & get the train/test/validation splits
2. Construct the model
3. Train the model on the train split
4. Evaluate the model predictions on the test split

### Dask - Scale our data loader to a planetary level!

Here we'll use Dask as the dataloader. Recall from the first notebook (*python_final_1_of_2_scikit_learn.ipynb*) that a data loader is a generator that loads data only when it is needed. This allows us to use datasets that are far too large to fit into the computer's memory. It can also allow us to use streaming data that comes in continuously from a source (*e.g.* instruments in the lab that are constantly measuring things).

In the first notebook we created a dataloader from scratch. However, typically you would use something like Dask to do this because we don't want to re-invent the wheel. (Note, there are other Python libraries that do out-of-memory and streaming/incremental modeling).


### Validation versus Test Data Splits

There are 2 datasets that are typically "held back" from training: the test dataset and the validation dataset.

"Held back" means that this data is never seen by the model during training. Before we begin training we split the data (randomly) into train, test, and validation shards. Only the training shard is used for training. This is the only way to provide an unbiased estimate of the model's performance. Remember, we are fitting the model to the training dataset so it stands to reason that the model will perform very well on the training dataset. 

The model is never trained on the validation nor test shards. The validation dataset is used to give an unbiased estimate of the performance of the final tuned model when comparing or selecting between final models. The test dataset is used at the very end (once you select the best model) to give an unbiased estimate of how your best model will perform with new data.

## Let's create a new classifier dataset and save it to a CSV file

We'll use the [Iris](https://archive.ics.uci.edu/ml/datasets/iris) dataset which is a classic classifier dataset to predict the type of flower from its petal measurements. The dataset fits into memory, but we're going to pretend that it doesn't and load it with Dask. This will exactly match the realworld case when we have GB to TB sized CSV files.

In [1]:
import pandas as pd

In [2]:
csv_filename = "iris_classification_dataset.csv"
target_label = "Flower class"

In [3]:
from sklearn import datasets

data = datasets.load_iris()
df = pd.DataFrame(data=data["data"], columns=data["feature_names"])
df[target_label] = data["target"]
df.to_csv(csv_filename, sep=",", index=False)

## Now we have our CSV. Next step is using Dask to load it.

Note we're using Dask to replace the standard Pandas library. Think of Dask as Pandas on steroids.

In [4]:
%pip install dask-ml[complete]   # install all optional dependencies

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
import dask.dataframe as dd
from dask_ml.model_selection import train_test_split

## Construct the model code


In [6]:
ddf = dd.read_csv(csv_filename, blocksize="16MB")

In [7]:
ddf

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),Flower class
npartitions=1,,,,,
,float64,float64,float64,float64,int64
,...,...,...,...,...


In [8]:
ddf.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),Flower class
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0


In [9]:
y = ddf[target_label]
X = ddf[ddf.columns.difference([target_label])]

X_train, X_remain, y_train, y_remain = train_test_split(
    X, y, test_size=0.3, shuffle=True, random_state=816
)

X_validate, X_test, y_validate, y_test = train_test_split(
    X_remain, y_remain, test_size=0.5, shuffle=True, random_state=816
)

In [10]:
y_train.head()

5      0
89     1
140    2
127    2
22     0
Name: Flower class, dtype: int64

In [11]:
X_train.head()

,petal length (cm),petal width (cm),sepal length (cm),sepal width (cm)
5,1.7,0.4,5.4,3.9
89,4.0,1.3,5.5,2.5
140,5.6,2.4,6.7,3.1
127,4.9,1.8,6.1,3.0
22,1.0,0.2,4.6,3.6


In [12]:
from tqdm.notebook import tqdm  # This gives us the nice progress bar!

## Time for the classifier model!

We're using the stochastic gradient descent classifier from scikit-learn. Note not all scikit-learn models will work with Dask. You'll need to find scikit-learn models that support the `partial_fit()` function. This function allows us to train one batch at a time.

In [13]:
from sklearn.linear_model import SGDClassifier
from dask_ml.wrappers import Incremental

In [14]:
classifier_model = SGDClassifier(loss="log_loss", verbose=True)  # Log loss is for classification models (use MSE for regression models)

## Normal scikit-learn with Dask

In the usual case, we can just use the `.fit()` method to train our model on our training data.

Just a single line!

In [15]:
classifier_model.fit(X_train, y_train)

-- Epoch 1
Norm: 75.52, NNZs: 4, Bias: 4.950373, T: 116, Avg. loss: 8.441691
Total training time: 0.00 seconds.
-- Epoch 2
Norm: 68.40, NNZs: 4, Bias: 4.950407, T: 232, Avg. loss: 0.000000
Total training time: 0.00 seconds.
-- Epoch 3
Norm: 62.51, NNZs: 4, Bias: 4.950477, T: 348, Avg. loss: 0.000000
Total training time: 0.00 seconds.
-- Epoch 4
Norm: 57.56, NNZs: 4, Bias: 4.950610, T: 464, Avg. loss: 0.000000
Total training time: 0.00 seconds.
-- Epoch 5
Norm: 53.33, NNZs: 4, Bias: 4.950783, T: 580, Avg. loss: 0.000000
Total training time: 0.00 seconds.
-- Epoch 6
Norm: 49.68, NNZs: 4, Bias: 4.951066, T: 696, Avg. loss: 0.000000
Total training time: 0.01 seconds.
-- Epoch 7
Norm: 46.50, NNZs: 4, Bias: 4.951371, T: 812, Avg. loss: 0.000000
Total training time: 0.01 seconds.
Convergence after 7 epochs took 0.01 seconds
-- Epoch 1
Norm: 112.88, NNZs: 4, Bias: 13.263940, T: 116, Avg. loss: 135.907970
Total training time: 0.00 seconds.
-- Epoch 2
Norm: 142.04, NNZs: 4, Bias: 13.222401, T: 2

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.1s finished


SGDClassifier(loss='log_loss', verbose=True)

## Model performance

Let's see how accurate our model is.

In [16]:
print(f"The final model has an accuracy of {100*classifier_model.score(X_test, y_test):.1f}%")

The final model has an accuracy of 78.3%


In [17]:
df_scikit_learn = pd.DataFrame()

df_scikit_learn["Truth"] = y_test
df_scikit_learn["Prediction"] =  classifier_model.predict(X_test)

In [18]:
def map_values(x:int):
    """Map the class value to the class name
    
    Args:
        x(int): Class value
    """
    if x == 0:
        return "Iris Setosa"
    elif x == 1:
        return "Iris Virginica"
    elif x == 2:
        return "Iris Versicolour"
    else:
        return "Error - Class not known"

In [19]:
df_scikit_learn["Truth"] = df_scikit_learn["Truth"].map(map_values)
df_scikit_learn["Prediction"] = df_scikit_learn["Prediction"].map(map_values)

df_scikit_learn["Match?"] = (df_scikit_learn["Truth"] == df_scikit_learn["Prediction"])

df_scikit_learn

,Truth,Prediction,Match?
0,Iris Setosa,Iris Setosa,True
1,Iris Versicolour,Iris Virginica,False
2,Iris Setosa,Iris Setosa,True
3,Iris Setosa,Iris Setosa,True
4,Iris Versicolour,Iris Virginica,False
5,Iris Setosa,Iris Setosa,True
6,Iris Virginica,Iris Virginica,True
7,Iris Setosa,Iris Virginica,False
8,Iris Setosa,Iris Setosa,True
9,Iris Setosa,Iris Setosa,True


In [20]:
classifier_model.coef_

array([[ -37.8232308 ,  -16.56516807,    2.21273765,   21.26105651],
       [  23.10204996, -103.2583644 ,   49.97657526, -118.83214136],
       [ 149.51635936,  146.0043028 , -107.23833363,  -76.5865024 ]])

## Incremental/batch/stream learning

So far everything has looked like our usual scikit-learn training. Here's where we tweak it a bit to allow for incremental learning. This might also be called batch learning or stream learning or online learning. It can handle datasets that are read in a batch at a time. This is wonderful when you are using really big datasets and datasets that are streaming from a source (like an IOT device).

In [21]:
classifier_model2 = SGDClassifier(loss="log_loss", verbose=True)  # Log loss is for classification models (use MSE for regression models)

In [22]:
classifier_increment = Incremental(classifier_model2, scoring="accuracy") 

In [23]:
NUMBER_OF_EPOCHS = 1000  # An epoch is just one pass through the dataset. We train for multiple epochs when we do SGD

In [24]:
classes = y_train.unique().compute()   # Note we need the .compute() at the end when using Dask (this is different than Pandas)
print(f"We have the following classes in our dataset:\n{classes}")

We have the following classes in our dataset:
0    0
1    1
2    2
Name: Flower class, dtype: int64


In [25]:
for _ in (pbar := tqdm(range(NUMBER_OF_EPOCHS))):
    
    # We're training for one batch at a time
    classifier_increment.partial_fit(X_train, y_train, classes=classes)
    
    pbar.set_description(f"Accuracy = {classifier_increment.score(X_train, y_train):.3f}")

  0%|          | 0/1000 [00:00<?, ?it/s]

-- Epoch 1
Norm: 134.77, NNZs: 4, Bias: 13.415669, T: 116, Avg. loss: 12.044180
Total training time: 0.00 seconds.
-- Epoch 1
Norm: 66.93, NNZs: 4, Bias: -0.726837, T: 116, Avg. loss: 127.130018
Total training time: 0.00 seconds.
-- Epoch 1
Norm: 231.00, NNZs: 4, Bias: -42.042006, T: 116, Avg. loss: 111.491819
Total training time: 0.00 seconds.
-- Epoch 1
Norm: 122.07, NNZs: 4, Bias: 13.415660, T: 116, Avg. loss: 0.000000
Total training time: 0.00 seconds.
-- Epoch 1
Norm: 98.17, NNZs: 4, Bias: 7.605255, T: 116, Avg. loss: 108.158837
Total training time: 0.00 seconds.
-- Epoch 1
Norm: 264.58, NNZs: 4, Bias: -59.445120, T: 116, Avg. loss: 44.270304
Total training time: 0.00 seconds.
-- Epoch 1
Norm: 111.56, NNZs: 4, Bias: 13.415567, T: 116, Avg. loss: 0.000000
Total training time: 0.00 seconds.
-- Epoch 1
Norm: 130.39, NNZs: 4, Bias: 9.169285, T: 116, Avg. loss: 82.655037
Total training time: 0.00 seconds.
-- Epoch 1
Norm: 301.65, NNZs: 4, Bias: -75.971964, T: 116, Avg. loss: 39.695857


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s finished


-- Epoch 1
Norm: 95.17, NNZs: 4, Bias: 13.407874, T: 116, Avg. loss: 0.000008
Total training time: 0.00 seconds.
-- Epoch 1
Norm: 175.75, NNZs: 4, Bias: 30.518850, T: 116, Avg. loss: 82.886675
Total training time: 0.00 seconds.
-- Epoch 1
Norm: 319.07, NNZs: 4, Bias: -89.054252, T: 116, Avg. loss: 20.340412
Total training time: 0.00 seconds.
-- Epoch 1
Norm: 88.66, NNZs: 4, Bias: 13.386922, T: 116, Avg. loss: 0.000030
Total training time: 0.00 seconds.
-- Epoch 1
Norm: 174.26, NNZs: 4, Bias: 42.726397, T: 116, Avg. loss: 89.474788
Total training time: 0.00 seconds.
-- Epoch 1
Norm: 324.73, NNZs: 4, Bias: -100.776265, T: 116, Avg. loss: 26.915020
Total training time: 0.00 seconds.
-- Epoch 1
Norm: 82.99, NNZs: 4, Bias: 13.358951, T: 116, Avg. loss: 0.000043
Total training time: 0.00 seconds.
-- Epoch 1
Norm: 185.24, NNZs: 4, Bias: 41.994887, T: 116, Avg. loss: 87.512854
Total training time: 0.00 seconds.
-- Epoch 1
Norm: 325.35, NNZs: 4, Bias: -106.461825, T: 116, Avg. loss: 14.551588
T

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s finished


...

-- Epoch 1
Norm: 5.81, NNZs: 4, Bias: 12.139211, T: 116, Avg. loss: 0.000584
Total training time: 0.00 seconds.
-- Epoch 1
Norm: 9.22, NNZs: 4, Bias: 17.517882, T: 116, Avg. loss: 1.209623
Total training time: 0.00 seconds.
-- Epoch 1
Norm: 47.27, NNZs: 4, Bias: -163.160996, T: 116, Avg. loss: 0.178981
Total training time: 0.00 seconds.
-- Epoch 1
Norm: 5.81, NNZs: 4, Bias: 12.139032, T: 116, Avg. loss: 0.000586
Total training time: 0.00 seconds.
-- Epoch 1
Norm: 8.69, NNZs: 4, Bias: 17.751178, T: 116, Avg. loss: 1.110733
Total training time: 0.00 seconds.
-- Epoch 1
Norm: 47.26, NNZs: 4, Bias: -163.144409, T: 116, Avg. loss: 0.222761
Total training time: 0.00 seconds.
-- Epoch 1
Norm: 5.81, NNZs: 4, Bias: 12.139062, T: 116, Avg. loss: 0.000585
Total training time: 0.00 seconds.
-- Epoch 1
Norm: 8.87, NNZs: 4, Bias: 17.601870, T: 116, Avg. loss: 1.178106
Total training time: 0.00 seconds.
-- Epoch 1
Norm: 47.16, NNZs: 4, Bias: -163.155243, T: 116, Avg. loss: 0.173763
Total training tim

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s finished


-- Epoch 1
Norm: 5.81, NNZs: 4, Bias: 12.138715, T: 116, Avg. loss: 0.000588
Total training time: 0.00 seconds.
-- Epoch 1
Norm: 8.72, NNZs: 4, Bias: 17.666856, T: 116, Avg. loss: 1.016119
Total training time: 0.00 seconds.
-- Epoch 1
Norm: 47.20, NNZs: 4, Bias: -163.113479, T: 116, Avg. loss: 0.227611
Total training time: 0.00 seconds.
-- Epoch 1
Norm: 5.81, NNZs: 4, Bias: 12.138716, T: 116, Avg. loss: 0.000585
Total training time: 0.00 seconds.
-- Epoch 1
Norm: 8.81, NNZs: 4, Bias: 17.625622, T: 116, Avg. loss: 1.051614
Total training time: 0.00 seconds.
-- Epoch 1
Norm: 46.98, NNZs: 4, Bias: -163.158556, T: 116, Avg. loss: 0.143344
Total training time: 0.00 seconds.
-- Epoch 1
Norm: 5.81, NNZs: 4, Bias: 12.138431, T: 116, Avg. loss: 0.000587
Total training time: 0.00 seconds.
-- Epoch 1
Norm: 8.86, NNZs: 4, Bias: 17.540077, T: 116, Avg. loss: 1.037222
Total training time: 0.00 seconds.
-- Epoch 1
Norm: 46.99, NNZs: 4, Bias: -163.139936, T: 116, Avg. loss: 0.194950
Total training tim

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s finished


-- Epoch 1
Norm: 5.81, NNZs: 4, Bias: 12.138313, T: 116, Avg. loss: 0.000584
Total training time: 0.00 seconds.
-- Epoch 1
Norm: 8.87, NNZs: 4, Bias: 17.479047, T: 116, Avg. loss: 1.020137
Total training time: 0.00 seconds.
-- Epoch 1
Norm: 46.72, NNZs: 4, Bias: -163.187251, T: 116, Avg. loss: 0.089994
Total training time: 0.00 seconds.
-- Epoch 1
Norm: 5.81, NNZs: 4, Bias: 12.138119, T: 116, Avg. loss: 0.000587
Total training time: 0.00 seconds.
-- Epoch 1
Norm: 8.75, NNZs: 4, Bias: 17.468603, T: 116, Avg. loss: 1.201374
Total training time: 0.00 seconds.
-- Epoch 1
Norm: 46.84, NNZs: 4, Bias: -163.134084, T: 116, Avg. loss: 0.245359
Total training time: 0.00 seconds.
-- Epoch 1
Norm: 5.81, NNZs: 4, Bias: 12.138172, T: 116, Avg. loss: 0.000585
Total training time: 0.00 seconds.
-- Epoch 1
Norm: 8.69, NNZs: 4, Bias: 17.456266, T: 116, Avg. loss: 1.132962
Total training time: 0.00 seconds.
-- Epoch 1
Norm: 47.43, NNZs: 4, Bias: -162.949130, T: 116, Avg. loss: 0.216466
Total training tim

## Check how well we did

During training we printed out the accuracy on the training dataset.

We use the test dataset to test how well the model performs to data that it hasn't seen before. This gives us an unbiased (hopefully) estimate of the true model performance for future data. This is what you would report as the final model accuracy.

In [26]:
print(f"The final model has an accuracy of {100*classifier_increment.score(X_test, y_test):.1f}%")

The final model has an accuracy of 100.0%


## Let's make some predictions of our test dataset

In [27]:
classifier_increment.predict(X_test)

dask.array<_predict, shape=(nan,), dtype=int64, chunksize=(nan,), chunktype=numpy.ndarray>

In [28]:
df_incremental = pd.DataFrame()

df_incremental["Truth"] = y_test.compute()  # Note the .compute()
df_incremental["Prediction"] =  classifier_increment.predict(X_test).compute()  # Note the .compute()

df_incremental

,Truth,Prediction
11,0,0
130,2,2
28,0,0
21,0,0
125,2,2
37,0,0
84,1,1
9,0,0
19,0,0
33,0,0


In [29]:
df_incremental["Truth"] = y_test.compute()  # Note the .compute()
df_incremental["Prediction"] =  classifier_increment.predict(X_test).compute()  # Note the .compute()


df_incremental["Truth"] = df_incremental["Truth"].map(map_values)
df_incremental["Prediction"] = df_incremental["Prediction"].map(map_values)

df_incremental["Match?"] = (df_incremental["Truth"] == df_incremental["Prediction"])

df_incremental

,Truth,Prediction,Match?
11,Iris Setosa,Iris Setosa,True
130,Iris Versicolour,Iris Versicolour,True
28,Iris Setosa,Iris Setosa,True
21,Iris Setosa,Iris Setosa,True
125,Iris Versicolour,Iris Versicolour,True
37,Iris Setosa,Iris Setosa,True
84,Iris Virginica,Iris Virginica,True
9,Iris Setosa,Iris Setosa,True
19,Iris Setosa,Iris Setosa,True
33,Iris Setosa,Iris Setosa,True


In [30]:
classifier_increment.coef_

array([[-4.82393941, -2.2657477 , -0.85024618,  2.1531378 ],
       [ 1.72177668, -4.88078173,  0.45017895, -6.97235262],
       [33.79125922, 32.28421395, -5.42551558, -6.00339025]])

In [31]:
classifier_increment.scoring

'accuracy'

## We won't have time to go over the validation dataset but it's a good topic for another time

Take a look at [hyperparameter optimization](https://scikit-learn.org/stable/modules/grid_search.html) for the details.

In [32]:
classifier_increment.score(X_validate, y_validate)

1.0